## Добавление индикаторов для нормализации

### 1. Стоимость фиксированного набора товаров и услуг

In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
from reg_normalizer.regions_validator import RegionMatcher

In [2]:
data = pd.read_excel('../data/external/data_fixed_basket.xls')
data = data.rename(columns={'Стоимость фиксированного набора потребительских товаров и услуг (рубль)': 'indicator_value'})

In [3]:
data['indicator_value'] = data['indicator_value'].str.replace('входящий в состав Пермского края', '', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('входящий в состав Камчатского края', '', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('Агинский Бурятский округ (Забайкальский край)', 'Агинский Бурятский округ', regex=False)

data['indicator_value'] = data['indicator_value'].str.replace('Тюменская область (без АО)', 'Тюменская область без автономных округов', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)', 'Тюменская область без автономных округов', regex=False)


data['indicator_value'] = data['indicator_value'].str.replace('Архангельская область (без АО)', 'Архангельская область без автономного округа', regex=False)
data['indicator_value'] = data['indicator_value'].str.replace('Архангельская область (кроме Ненецкого автономного округа)', 'Архангельская область без автономного округа', regex=False)

In [4]:
data = data.drop(index=0)
data = data.drop(index=2)
data.columns = data.iloc[0]
data = data.drop(index=1)
data = data.rename(columns={data.columns[0]: 'region'})
data.head()

1,region,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
3,Российская Федерация,3196.74,3577.03,4091.08,4709.74,5231.27,6068.25,6973.55,7714.12,8711.78,...,14212.11,14789.42,15467.92,16067.38,17052.67,18728.18,20224.9,NaN,NaN,NaN
4,Центральный федеральный округ,3329.09,3777.66,4339.65,5059.91,5616.82,6522.73,7502.09,8369.62,9470.95,...,15769.38,16678.92,17416.17,18537.87,19774.85,21603.35,22487.14,24275.42,26874.87,29474.36
5,Белгородская область,2747.37,3035.4,3538.49,4077.86,4544.95,5234.31,5954.31,6556.24,7286.67,...,12526.77,12933.25,13804.25,14110.96,14972.94,16022.72,17590.17,18438.06,20570.47,22338.71
6,Брянская область,2764.37,3037.41,3482.91,4006.58,4417.29,5185.99,5952.28,6685.62,7542.59,...,13293.24,13807.54,14504.38,14915.02,15612.58,17089.49,19294.32,20420.06,22168.65,23924.09
7,Владимирская область,2568.1,3005.14,3558.76,4125.27,4685.66,5683.05,6722.62,7257.76,8404.94,...,14056.93,14555.07,15331.3,15742.5,16586.44,17909.47,20134.67,21014.01,23939.4,26387.58


In [5]:
data['region'] = data['region'].str.lstrip()

In [6]:
matcher = RegionMatcher()

In [8]:
data = matcher.attach_fields(data, 'region', ['name_rus', 'oktmo'], threshold=0.5)

In [9]:
data.to_excel('../data/interim/data_fixed_basket_check.xlsx')

In [10]:
years = list(range(2000, 2026))

# Normalize year column names to strings so we can reliably melt.
rename_map = {}
for y in years:
    if y in data.columns:
        rename_map[y] = str(y)

temp_data = data.rename(columns=rename_map).copy()

for y in years:
    col = str(y)
    if col not in temp_data.columns:
        temp_data[col] = pd.NA

year_cols = [str(y) for y in years]

# Create new DataFrame for output
long_data = temp_data[['name_rus', 'oktmo', *year_cols]].melt(
    id_vars=['name_rus', 'oktmo'],
    value_vars=year_cols,
    var_name='year',
    value_name='indicator_value',
)

long_data['year'] = long_data['year'].astype(int)
# Sort data by name_rus and year (ascending)
long_data = long_data.sort_values(by=['name_rus', 'year']).reset_index(drop=True)

In [12]:
long_data.to_excel('../data/processed/fixed_basket_final.xlsx')